In [1]:
import sys
import pandas as pd
sys.path.append("..")

from chemspace.Dataset.DatasetBuilder import DatasetBuilder

Let's get the synonyms into one column of strings, although they must not be comma-separated, to avoid problems later in the week

Let's find the data

In [2]:
# Instantiate class with previously generated Dataframe (CSV)
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/Dataset.csv')
#DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/CIDs.csv')
# Display dataset
DB.CIDs
DB.dataset

,CID,SMILES
0,1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
1,3,C1=CC(C(C(=C1)C(=O)O)O)O
2,4,CC(CN)O
3,5,C(C(=O)COP(=O)(O)O)N
4,6,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl
...,...,...
327752,168010365,CC[C@@H](C)[C@H](C(=O)N1[C@H]2C[C@@H](CC[C@H]2...
327753,168010381,CCCC/C=C/C=C/CC1C(CC(=O)C2=C(C(=C)OC(=O)C2(O1)...
327754,168010385,CCCCC[C@@H]1CC(=NN(O1)C(=O)CCOC(=O)[C@H](C)OC(...
327755,168010395,CC[C@@H](C)[C@H](C(=O)N1[C@H]2C[C@@H](CC[C@H]2...


In [3]:
import gzip

file_path = '../chemspace/Dataset/Data/CID-Synonym-filtered.gz'  # Replace with the path to your .gz file

with gzip.open(file_path, 'rb') as f:
    chunk = f.read(2048)  # Read the first 1024 bytes

    # Process the chunk as needed
    # Example: Perform calculations, write to another file, etc.
    print(chunk)  # Print the chunk content as an example



b'1\t(+/-)-acetylcarnitine\n1\t(3-CARBOXY-2-HYDROXYPROPYL)TRIMETHYLAMMONIUM HYDROXIDE INNER SALT ACETATE\n1\t07OP6H4V4A\n1\t1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-trimethyl-, hydroxide, inner salt, (+)-\n1\t1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-trimethyl-, hydroxide, inner salt, (+-)-\n1\t1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-trimethyl-, hydroxide, inner salt, (+-)- (9CI)\n1\t1-Propanaminium, 2-(acetyloxy)-3-carboxy-N,N,N-trimethyl-, inner salt\n1\t14992-62-2\n1\t2-(ACETYLOXY)-3-CARBOXY-N,N,N-TRIMETHYL-1-PROPANAMINIUM INNER SALT\n1\t3-(acetyloxy)-4-(trimethylammonio)butanoate\n1\t3-(acetyloxy)-4-(trimethylazaniumyl)butanoate\n1\t3-acetyloxy-4-(trimethylazaniumyl)butanoate\n1\t870-77-9\n1\tAcetyl carnitine\n1\tAcetyl-DL-carnitine\n1\tacetylcarnitine\n1\tACETYLCARNITINE DL-FORM\n1\tACETYLCARNITINE RACEMATE [MI]\n1\tACETYLCARNITINE, (+/-)-\n1\tACETYLCARNITINE, DL-\n1\tAMMONIUM, (3-CARBOXY-2-HYDROXYPROPYL)TRIMETHYL-, HYDROXIDE, INNER SALT, ACETATE\n1\tAmmonium, (3-carbo

In [17]:
import gzip
import pandas as pd
import io

bytes_to_read = 10**7  # Number of bytes to read

# Read the first 2048 bytes from the gzipped file
with gzip.open(file_path, 'rb') as f:
    unzipped_data = f.read(bytes_to_read).decode('utf-8')  # Adjust the encoding if needed

# Save the unzipped data to a new file
output_file_path = 'unzipped_data.txt'  # Replace with the desired output file path
with open(output_file_path, 'w') as f:
    f.write(unzipped_data)

print(f"Unzipped data saved to: {output_file_path}")


Unzipped data saved to: unzipped_data.txt


In [11]:
data = unzipped_data

lines = data.strip().split('\n')
cids = []
names = []

for line in lines:
    ref_num, name = line.split('\t')
    cids.append(int(ref_num)) # convert to integers
    names.append(name)

# Create a list with names grouped by reference numbers
list_result = []
current_cid = None
current_names = []

for cid, name in zip(cids, names):
    if ref_num != current_cid:
        if current_cid:
            list_result.append(' '.join(current_names)) # adds a space between names
            # list_result.append('; '.join(current_names))  # adds a semicolon between names
        current_cid = cid
        current_names = [name]
    else:
        current_names.append(name)

# Append the last group of names
if current_cid:
    list_result.append(' '.join(current_names))

print("List Result:")
print(list_result)

# Create a DataFrame with names grouped by reference numbers
df = pd.DataFrame({'CID': cids, 'Names': names})
df_grouped = df.groupby('CID')['Names'].apply(' '.join).reset_index()

print("\nDataFrame Result:")
print(df_grouped)


List Result:
['(+/-)-acetylcarnitine', '(3-CARBOXY-2-HYDROXYPROPYL)TRIMETHYLAMMONIUM HYDROXIDE INNER SALT ACETATE', '07OP6H4V4A', '1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-trimethyl-, hydroxide, inner salt, (+)-', '1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-trimethyl-, hydroxide, inner salt, (+-)-', '1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-trimethyl-, hydroxide, inner salt, (+-)- (9CI)', '1-Propanaminium, 2-(acetyloxy)-3-carboxy-N,N,N-trimethyl-, inner salt', '14992-62-2', '2-(ACETYLOXY)-3-CARBOXY-N,N,N-TRIMETHYL-1-PROPANAMINIUM INNER SALT', '3-(acetyloxy)-4-(trimethylammonio)butanoate', '3-(acetyloxy)-4-(trimethylazaniumyl)butanoate', '3-acetyloxy-4-(trimethylazaniumyl)butanoate', '870-77-9', 'Acetyl carnitine', 'Acetyl-DL-carnitine', 'acetylcarnitine', 'ACETYLCARNITINE DL-FORM', 'ACETYLCARNITINE RACEMATE [MI]', 'ACETYLCARNITINE, (+/-)-', 'ACETYLCARNITINE, DL-', 'AMMONIUM, (3-CARBOXY-2-HYDROXYPROPYL)TRIMETHYL-, HYDROXIDE, INNER SALT, ACETATE', 'Ammonium, (3-carboxy-2-hydrox

In [16]:
data = unzipped_data

lines = data.strip().split('\n')
cids = []
synonyms = []

for line in lines:
    cid, synonym = line.split('\t')
    cids.append(int(cid))
    synonyms.append(synonym)

# Create a list with synonyms grouped by reference numbers
list_result = []
current_cid = None
current_synonyms = []

for cid, synonym in zip(cids, synonyms):
    if cid != current_cid:
        if current_cid:
        #     list_result.append({'CID': current_cid, 'Synonyms': '; '.join(current_synonyms),
        #                         'Number of Synonyms': len(current_synonyms)}) # adds a semicolon between synonyms
            list_result.append({'CID': current_cid, 'Synonyms': '\n'.join(current_synonyms),
                                'Number of Synonyms': len(current_synonyms)}) # adds a new line between synonyms
            # list_result.append({'CID': current_cid, 'Synonyms': ' '.join(current_synonyms),
                                # 'Number of Synonyms': len(current_synonyms)}) # adds a space between synonyms
        current_cid = cid
        current_synonyms = [synonym]
    else:
        current_synonyms.append(synonym)

# Append the last group of synonyms
if current_cid:
    list_result.append({'CID': current_cid, 'Synonyms': '; '.join(current_synonyms),
                        'Number of Synonyms': len(current_synonyms)})

# Append the last group of synonyms
if current_cid:
    list_result.append({'CID': current_cid, 'Synonyms': '; '.join(current_synonyms),
                        'Number of Synonyms': len(current_synonyms)})

# print("List Result:")
# for item in list_result:
#     print(item)

# Create a DataFrame with synonyms grouped by reference numbers
df = pd.DataFrame({'CID': cids, 'Synonyms': synonyms})
df_grouped = df.groupby('CID').agg(Synonyms=('Synonyms', lambda x: '; '.join(x)),
                                               Number_of_Synonyms=('Synonyms', 'count')).reset_index()

# print("\nDataFrame Result:")
# print(df_grouped)
df_grouped.head()


,CID,Synonyms,Number_of_Synonyms
0,1,(+/-)-acetylcarnitine; (3-CARBOXY-2-HYDROXYPRO...,38
1,2,[2-(ACETYLOXY)-3-CARBOXYPROPYL]TRIMETHYLAZANIU...,13
2,3,"100459-00-5; 2,3-dihydro-2,3-dihydroxybenzoic ...",9
3,4,(+-)-1-amino-2-propanol; (+/-)-1-Amino-2-propa...,122
4,5,(3-amino-2-oxopropyl) dihydrogen phosphate; 1-...,11


In [7]:
# save dataframe to csv
df_grouped.to_csv('../chemspace/Dataset/Data/CID-Synonym-filtered-test.csv', index=False)


In [3]:
def add_SMILES(self, data_path: str = '../chemspace/Dataset/Data/CID-SMILES.gz'):
        concat_df = pd.DataFrame()
        
        # create object to iterate through CSV chunks
        data_reader = pd.read_csv(data_path, chunksize= 10 ** 6, sep= "\t", names = ['CID', 'SMILES'])
        
        # Initialize counter and 
        i = 0

        # Iterate through df chunks
        for df in data_reader:
            # Display progress
            if i % 5 == 0:
                print(i)

            # if the lowest CID of the chunk is greater than the largest CID we're interested in, then stop
            if df['CID'].iloc[0] > self.CIDs.iloc[-1]:
                return
            # Otherwise, merge the CIDs and SMILES chunk dataframes, and concat. Store as dataset
            elif self._external_CIDs_in_dataset(df['CID']):
                merged_df = self.CIDs.to_frame().merge(df, how='inner', left_on = 'CID', right_on='CID', suffixes= (None,'_y'))
                concat_df = pd.concat([concat_df, merged_df], axis = 0, ignore_index=True)
            
            # Advance counter
            i = i + 1
            self.dataset = concat_df
        return

In [4]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58001485 entries, 0 to 58001484
Data columns (total 3 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   CID                 int64 
 1   Synonyms            object
 2   Number_of_Synonyms  int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ GB


In [5]:
# Load df
df = pd.read_csv('../chemspace/Dataset/Data/Dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327757 entries, 0 to 327756
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   CID     327757 non-null  int64 
 1   SMILES  327757 non-null  object
dtypes: int64(1), object(1)
memory usage: 5.0+ MB


In [6]:

# Assuming df and df_grouped are your dataframes

# Initialize an empty dataframe to store the final result
result = pd.DataFrame()

# Number of rows to process at a time
chunk_size = 1000

# Iterate through the df_grouped dataframe in chunks
for i in range(0, len(df_grouped), chunk_size):
    # Get the current chunk from df_grouped
    chunk = df_grouped.iloc[i:i + chunk_size]
    
    # Merge the chunk with df based on CID
    merged_chunk = df.merge(chunk, how='inner', on='CID')
    
    # Append the merged chunk to the result dataframe
    result = pd.concat([result, merged_chunk], ignore_index=True)

# result dataframe now contains the merged data
print(result)


              CID                                             SMILES  \
0               1                   CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C   
1               3                           C1=CC(C(C(=C1)C(=O)O)O)O   
2               4                                            CC(CN)O   
3               5                               C(C(=O)COP(=O)(O)O)N   
4               6          C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl   
...           ...                                                ...   
250092  168010365  CC[C@@H](C)[C@H](C(=O)N1[C@H]2C[C@@H](CC[C@H]2...   
250093  168010381  CCCC/C=C/C=C/CC1C(CC(=O)C2=C(C(=C)OC(=O)C2(O1)...   
250094  168010385  CCCCC[C@@H]1CC(=NN(O1)C(=O)CCOC(=O)[C@H](C)OC(...   
250095  168010395  CC[C@@H](C)[C@H](C(=O)N1[C@H]2C[C@@H](CC[C@H]2...   
250096  168011857  COC1=CC(=CC2=C1C3=CC4=C(C=C3CC2)O[C@H]([C@@H]4...   

                                                 Synonyms  Number_of_Synonyms  
0       (+/-)-acetylcarnitine| (3-CARBOXY-2-HYDROXYPRO.

In [7]:
#write result to csv
result.to_csv('../chemspace/Dataset/Data/Dataset-with-Synonyms.csv', index=False)

In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250097 entries, 0 to 250096
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   CID                 250097 non-null  int64 
 1   SMILES              250097 non-null  object
 2   Synonyms            250097 non-null  object
 3   Number_of_Synonyms  250097 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 7.6+ MB


In [9]:
result.head()

,CID,SMILES,Synonyms,Number_of_Synonyms
0,1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,(+/-)-acetylcarnitine| (3-CARBOXY-2-HYDROXYPRO...,38
1,3,C1=CC(C(C(=C1)C(=O)O)O)O,"100459-00-5| 2,3-dihydro-2,3-dihydroxybenzoic ...",9
2,4,CC(CN)O,(+-)-1-amino-2-propanol| (+/-)-1-Amino-2-propa...,122
3,5,C(C(=O)COP(=O)(O)O)N,(3-amino-2-oxopropyl) dihydrogen phosphate| 1-...,11
4,6,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"1 - Chloro - 2,4 - dinitrobenzene| 1,3-Dinitro...",101
